In [ ]:
import csv
import json
import re

ORIGINATOR = "24273"#"61746" #692484" #city
#ORIGINATOR = "73981" #Bank of America
INPUT = "./2025-11-13_on_boarding_messages/2025-11-13/com.att.aamg.kafka.prod.sms-prod-eastus2-mt-default.csv" 
OUTPUT = f"./data/input/ref_{ORIGINATOR}.csv"
MAX_ROWS = 1000000
DILUTE_FACTOR = 10  # write every Nth message


def normalize(s):
    s = s.strip()
    if len(s) >= 2 and s[0] == '"' and s[-1] == '"':
        s = s[1:-1]
    s = s.replace('""', '"').replace('\r', ' ').replace('\n', ' ').replace('\t', ' ')
    return s

def get_source_and_fields(raw):
    s = normalize(raw)
    try:
        o = json.loads(s)
    except json.JSONDecodeError:
        # try simple heuristic extraction
        src = re.search(r'"sourceAddress"\s*:\s*"([^"]*)"', s)
        req = re.search(r'"requestID"\s*:\s*"([^"]*)"', s)
        msg = re.search(r'"shortMessage"\s*:\s*"([^"]*)"', s)
        return (src.group(1) if src else None,
                req.group(1) if req else None,
                msg.group(1).replace('\\"','"') if msg else None)
    return (o.get('sourceAddress'), o.get('requestID'), o.get('shortMessage'))


with open(INPUT, newline='', encoding='utf-8') as fin, open(OUTPUT, 'w', newline='', encoding='utf-8') as fout:
    reader = csv.DictReader(fin)
    writer = csv.DictWriter(fout, fieldnames=['originator_id','message_id','raw_text'])
    writer.writeheader()
    for i, row in enumerate(reader, 1):
        src, req, msg = get_source_and_fields(row['Value'])
        if src == ORIGINATOR and i % DILUTE_FACTOR == 0:
            writer.writerow({'originator_id': src, 'message_id': req, 'raw_text': msg})
        if i % 100000 == 0:
            print(f"Processed {i:,} rows")
        if i >= MAX_ROWS:
            break


Processed 100,000 rows
Processed 200,000 rows
Processed 300,000 rows
Processed 400,000 rows
Processed 500,000 rows
Processed 600,000 rows
Processed 700,000 rows
Processed 800,000 rows
Processed 900,000 rows
Processed 1,000,000 rows
